In [1]:
#import textract
#text = textract.process("C:/Users/Ranja.Sarkar/Desktop/new1/GS.05.51610_Polyethylene lined pipe systems.pdf")

#path = r"C:/Users/Ranja.Sarkar/Desktop/new1/GS.05.51610_Polyethylene lined pipe systems.pdf"

import PyPDF2

pdfFileObj = open('C:/Users/Ranja.Sarkar/Desktop/new3/GS.05.51610_Polyethylene lined pipe systems.pdf','rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

pageObj = pdfReader.getPage(9)
pagetxt = pageObj.extractText()
#print(pagetxt)

In [ ]:
## SPACY FOR POS TAGGING

"""

import spacy
#python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

doc = nlp(u"Apple is looking at buying U.K. startup for $1 billion")
for token in doc:
    print(token.text,token.lemma_,token.pos_,token.is_stop)
    
"""

In [6]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(pagetxt)
#doc = nlp("Environmental Stress Cracking (ESC) can best be described as the localized cleaving of portions of molecular chains in an area of concentrated stress. Fluids that will not usually attack a polymer in an unstressed state will attack the area weakened by localized stress causing a crack or craze.")

In [ ]:
"""
for sents in doc.sents:
#    print(len(sents.text))
    print(sents.text)
    
"""

In [ ]:
##words in the sentences, considered with POS tags only

"""
words_pos = ['NOUN','PROPN','VERB','ADJECTIVE','ADVERB']

sentences = []
for sent in doc.sents:
    selected_words = []
    for token in sent:
        if token.pos_ in words_pos and token.is_stop is False:
            selected_words.append(token)
    sentences.append(selected_words)
print(sentences)

"""

In [7]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

#nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 #damping coefficient
        self.min_diff = 1e-5 #convergence threshold
        self.steps = 20 #iteration steps, not 10
        self.node_weight = None # save keywords and its weight
        
        
    def set_stopwords(self,stopwords):
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
            
    def sentence_segment(self,doc,words_pos,lower):
        """Store words only in words_pos tag"""
        sentences=[]
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                if token.pos_ in words_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return(sentences)

    def get_vocab(self,sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""

        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)

        return token_pairs
    
    def symmetrize(self, a):

        return a + a.T - np.diag(a.diagonal())


    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix

        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
         

        # Get Symmeric matrix
        g = self.symmetrize(g)

        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is to ignore 0 element in norm

        return g_norm
    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
    
    def analyze(self, text, words_pos = ['NOUN', 'PROPN', 'VERB','ADJECTIVE','ADVERB'], window_size = 8, lower = False, stopwords = list()):
        
        """Main function to analyze text"""
      
        # Set stop words
        self.set_stopwords(stopwords)

        # Parse text by spaCy
        doc = nlp(text)

        # Filter sentences
        sentences = self.sentence_segment(doc, words_pos, lower) # list of list of words
     

        # Build vocabulary
        vocab = self.get_vocab(sentences)

        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)

        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)

        
        # Initialization for weight(pagerank value)
        pr = np.array([1] * len(vocab))

        # Iteration
        previous_pr = 0

        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:

                break
            else:
                previous_pr = sum(pr)


        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]

        self.node_weight = node_weight  

In [8]:
text = pagetxt
tr4w = TextRank4Keyword()
tr4w.analyze(text, words_pos = ['NOUN', 'PROPN','VERB','ADJECTIVE','ADVERB'], window_size=8, lower=False)
#tr4w.get_keywords(10)
kw = tr4w.get_keywords()
#table = pd.DataFrame(kw,columns=['weights','keywords'])
#table = table.sort_values('weights',ascending=False)
#table.head(10)

liner - 4.489944934035321
ESC - 3.285848979487741
lined - 3.2811066548079455
liners - 2.4939643830089597
failures - 2.4197970364333306
PE - 2.234195475859875
pipelines - 2.2236345716529633
Bar - 1.9498810491276397
collapse - 1.7734945969762803
Table - 1.6771637971173572
localized - 1.6130258908830464
Shell - 1.5956041715468787


In [ ]:
#import nltk
from rake_nltk import Rake
import pandas as pd

r = Rake()

words = r.extract_keywords_from_text(text)
#phrases = r.get_ranked_phrases()
phrases = r.get_ranked_phrases_with_scores()
table = pd.DataFrame(phrases,columns=['score','Key phrases'])
table = table.sort_values('score',ascending=False)
table.head(10)

In [2]:
## GENSIM FOR KEYWORD EXTRACTION

from gensim.summarization import keywords

import warnings
warnings.filterwarnings("ignore")
#import pandas as pd

text = pagetxt
values = keywords(text, split = '\n',scores = True)
#data = pd.DataFrame(values,columns=['Keywords','Score'])
#data = data.sort_values('Score',ascending=False)
#data.head(10)